In [1]:
from transformers import BertModel, BertTokenizer
import torch

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = '/hy-tmp'
tokernizer = BertTokenizer.from_pretrained(path)
model = BertModel.from_pretrained(path)

Some weights of the model checkpoint at /hy-tmp were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text = 'load model over'
tokernizer.tokenize(text)

['load', 'model', 'over']

In [4]:
ids = torch.LongTensor(tokernizer.convert_tokens_to_ids(tokernizer.tokenize(text)))
model(ids.unsqueeze(0))

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3482, -0.1659,  0.0453,  ..., -0.1100, -0.2788,  0.2880],
         [-0.2050,  0.3918, -0.0266,  ..., -0.1452, -0.0821,  0.6034],
         [-0.1642,  0.3814, -0.0294,  ..., -0.1611, -0.0450,  0.6710]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-2.6592e-01, -1.0810e-01,  9.4855e-01,  2.2788e-01, -2.3948e-01,
         -2.3168e-03,  2.8078e-01,  1.6776e-01,  9.1976e-01, -4.1347e-01,
          4.6883e-01, -5.3548e-01,  9.0963e-01, -8.7861e-01,  9.0639e-01,
         -3.4364e-01,  1.4422e-01, -1.1521e-01,  2.1253e-01, -5.3151e-01,
          8.3666e-01, -8.1899e-01,  5.7772e-01,  5.4666e-02, -8.1516e-02,
         -8.6221e-01, -8.1418e-02,  9.2704e-01,  7.5403e-01,  8.1938e-01,
         -7.3761e-01,  2.1425e-01, -9.6725e-01, -1.4284e-01,  8.8671e-01,
         -7.9322e-01, -9.3905e-02, -3.1478e-01, -7.8598e-02, -5.7400e-03,
         -8.9994e-01,  1.9232e-01, -3.2142e-02, -7.0389e-01,  1.8028e-01

In [5]:
!python3 Bertmodel2ONNX.py

pytorch: 1.11.0+cu113
transformers: 4.19.2
Some weights of the model checkpoint at /hy-tmp were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
==============model test===================
torch.Size([1, 16, 30522])
model test topk10 output:
The capital of France, paris, contains the Eiffel Tower.
The capital of France, lyon, contains the Eiffel Tower.
The capital of France, lille, contains the Eiffel Tower.
The capital of France, toulouse, contains the Eiffel Tower.
The cap

In [6]:
!onnxsim model.onnx model-sim.onnx --input-shape input_ids:1,12 token_type_ids:1,12 attention_mask:1,12 --dynamic-input-shape

Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add        │ 177            │ 177              │
│ Cast       │ 1              │ 1                │
│ Concat     │ 48             │ 48               │
│ Constant   │ 289            │ 0                │
│ Div        │ 51             │ 51               │
│ Erf        │ 13             │ 13               │
│ Gather     │ 100            │ 100              │
│ MatMul     │ 98             │ 98               │
│ Mul        │ 53             │ 53               │
│ Pow        │ 26             │ 26               │
│ ReduceMean │ 52             │ 52               │
│ Reshape    │ 48             │ 48               │
│ Shape      │ 97             │ 97               │
│ Slice      │ 1              │ 1                │
│ Softmax    │ 12             │ 12               │
│ Sqrt       │ 26             │ 26 

In [16]:
!python3 build_plan.py

Building model ...
[TensorRT] VERBOSE: Registered plugin creator - ::GridAnchor_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::NMS_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::Reorg_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::Region_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::Clip_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::LReLU_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::PriorBox_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::Normalize_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::RPROI_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::BatchedNMS_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::BatchedNMSDynamic_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::FlattenConcat_TRT version 1
[TensorRT] VERBOSE: Registered plugin creator - ::CropAndResize version 1
[TensorRT] VERBOSE: Registered p

In [17]:
!ls

Bertmodel2ONNX.py  Engine	    pytorch_model.bin	   vocab.txt
build_plan.py	   model_fp32.plan  test.ipynb
case_data.npz	   model.onnx	    tokenizer_config.json
config.json	   model-sim.onnx   tokenizer.json
